# Tier Wrapper generator

The following notebook generates the wrapping code to make C++ available to the pyclesperanto python code.
This notebook is to be runned for each Tiers (1,2,3, etc.) and will generate the corresponding wrapping code.
It only need to be runned when a Tier is updated.

__*WARNING*__: this can break the code if the python package is not update consequently. Please check the code and run the tests before pushing.

In [73]:
import glob
import os
import sys
import re
import numpy as np

# define __file__ if it is not defined
if '__file__' not in globals():
    __file__ = os.path.abspath('generate-tiers-package.ipynb')

# get platform linux, windows, darwin
platform = sys.platform
# get platform architecture x86_64, x86, armv7l
architecture = os.uname().machine
# get the python version
python_version = sys.version_info

Get the list of hpp files in the tiers directory and sort them by name.

In [74]:
# get first argument of the script
tier = 2 #sys.argv[1]

# get current directory
current_dir = os.path.dirname(os.path.abspath(__file__))
# get the relative path to directory and add 'tier' plus the number of the tier
tier_dir = os.path.join(current_dir, '..', '_skbuild', 'linux-x86_64-3.10', 'cmake-build', '_deps', 'clic_lib-src', 'clic', 'include', 'tier' + str(tier))
# list all files in the tier_dir
files = glob.glob(tier_dir + '.hpp')
print(files)

['/mnt/data/Clesperanto/repo/pyclesperanto/scripts/../_skbuild/linux-x86_64-3.10/cmake-build/_deps/clic_lib-src/clic/include/tier2.hpp']


Define a set of function to parse the hpp files and extract the information we need.

In [75]:
def read_file(file):
    with open(file, 'r') as f:
        content = f.read()
    return content

def parse_file(file_content):
    # Regular expression pattern to extract function names and parameter names
    pattern = r"(\w+_func)\s*\(([^)]*)\)"

    # Extract function names and parameter names using regular expression
    matches = re.findall(pattern, file_content, re.MULTILINE)

    # Create a dictionary to store function names and their corresponding parameter names
    function_parameters = {}

    # Process each match to extract function name and parameter names
    for match in matches:
        function_name, parameters = match
        parameter_names = [param.strip().split()[-1] for param in parameters.split(",")]
        function_parameters[function_name] = parameter_names
    
    return function_parameters


def generate_function(tier, function_name, parameter_names):
    # generate the function name for python and c++
    py_function_name = function_name.replace("_func", "")
    cpp_function_name = f"&cle::tier{tier}::{function_name}"
    # generate the function body for the wrapper
    wrapper_function = f"m.def(\"_{py_function_name}\", {cpp_function_name}, \"Call {py_function_name} from C++.\",\n"
    wrapper_function += "\t\tpy::return_value_policy::take_ownership,\n"
    wrapper_function += "\t\tpy::arg(\""+ parameter_names[0] +"\")"
    for parameter_name in parameter_names[1:]:
        wrapper_function += ", py::arg(\""
        wrapper_function += parameter_name
        wrapper_function += "\")"
    wrapper_function += ");"
    return wrapper_function


For each files, we extract the name of the function it contains as well as its signature. From both we build the wrapper code to make the function visible in Python.

In [76]:
wrapper_functions = []

file_content = read_file(files[0])
function_parameters = parse_file(file_content)
function_parameters = parse_file(file_content)

# Print the function names and their corresponding parameter names
for function_name, parameter_names in function_parameters.items():
    print(f"Function: {function_name}")
    print(f"Parameters: {parameter_names}")
    print()

for function_name, parameter_names in function_parameters.items():
    wrap = generate_function(tier, function_name, parameter_names)
    wrapper_functions.append(wrap)

# print each wrapper function on a new line 
for wrapper_function in wrapper_functions:
    print("\t",wrapper_function)

Function: difference_of_gaussian_func
Parameters: ['device', 'src', 'dst', 'sigma1_x', 'sigma1_y', 'sigma1_z', 'sigma2_x', 'sigma2_y', 'sigma2_z']

difference_of_gaussian_func difference_of_gaussian
	 m.def("_difference_of_gaussian", &cle::tier2::difference_of_gaussian_func, "Call difference_of_gaussian from C++.",
		py::return_value_policy::take_ownership,
		py::arg("device"), py::arg("src"), py::arg("dst"), py::arg("sigma1_x"), py::arg("sigma1_y"), py::arg("sigma1_z"), py::arg("sigma2_x"), py::arg("sigma2_y"), py::arg("sigma2_z"));


We create the wrapper file to contains the Tier functions currently processed. And for each function, we add the wrapper code to the file.

In [77]:
# create a cpp file for the current tier call cleTierX.cpp

# get the path to the current directory
current_dir = os.path.dirname(os.path.abspath(__file__))
# get the path to the directory where the cpp file should be stored
cpp_dir = os.path.join(current_dir, '..', 'wrapper')
# create the cpp file
cpp_file = os.path.join(cpp_dir, 'tier' + str(tier) + '_wrapper.cpp')

# if the file already exists, rename it to cleTierX_old.cpp
if os.path.exists(cpp_file):
    os.rename(cpp_file, cpp_file.replace('.cpp', '_old.cpp'))

with open(cpp_file, 'w') as f:
    # write the header for pyclesperanto
    f.write('#include "pyclesperanto.hpp"\n')
    f.write(f'#include "tier{tier}.hpp"\n')
    # skip a line
    f.write('\n')
    f.write('namespace py = pybind11;\n')
    f.write('\n')
    # write the module function
    f.write('auto wrapper_tier' + str(tier) + '(py::module_ &m) -> void\n{\n')
    # skip a line
    f.write('\n')
    # write the wrapper functions
    for wrapper_function in wrapper_functions:
        f.write('\t' + wrapper_function + '\n')
        f.write('\n')
    # close the module function
    f.write('}')